In [4]:
import pandas as pd
from datasets import load_dataset

polemo_category = "hotels_text"
polemo_official = load_dataset("data/polemo2-official/", polemo_category) # only oppinions about hotels
df_polemo_official = pd.DataFrame(polemo_official["train"])

100%|██████████| 3/3 [00:00<00:00, 930.34it/s]


In [5]:
import sys
sys.path.insert(0, './spellcorrectorpl/python')
from KnownWordsProvider import KnownWordsProviderUsingRAM, KnownWordsProviderUsingBigFile, KnownWordsProviderUsingMultipleFiles
from BigramsProvider import BigramsProvider
from SpellCorrector import SpellCorrector
import re

UNIGRAMS_FILEPATH = 'spellcorrectorpl/out/1grams_fixed'
UNIGRAMS_FILES_DIR = 'spellcorrectorpl/out/1grams_splitted/'
BIGRAMS_FILEPATH = "spellcorrectorpl/out/2grams_splitted"
 
# unigrams_path = UNIGRAMS_FILEPATH
words_provider = KnownWordsProviderUsingMultipleFiles()
bigrams_provider = BigramsProvider()
words_provider.initialize(UNIGRAMS_FILES_DIR)
bigrams_provider.initialize(BIGRAMS_FILEPATH)

corrector = SpellCorrector(words_provider, bigrams_provider)

def correct_opinion(opinion: str, corrector: SpellCorrector):        
    opinion = re.sub(r'\d', "<liczba>", opinion) #change numbers to #L
    opinion = corrector.sentence_correction(opinion, print_words=False)
    return opinion


In [6]:
from string import punctuation
import unidecode

def preprocess(df: pd.DataFrame):
    translate_table = dict((ord(char), None) for char in punctuation)
    oppinions = df.copy()
    oppinions["text"] = oppinions["text"].str.translate(translate_table) # remove punctuation
    # TODO: usuwać słowa w cudzysłowie
    oppinions["text"] = oppinions["text"].apply(correct_opinion, corrector=corrector)
    oppinions["text"] = oppinions["text"].str.casefold() # to lower
    oppinions["text"] = oppinions["text"].apply(unidecode.unidecode) # remove polish characters
    return oppinions

# test_df = pd.DataFrame([["ala ma hogel", 1], ["jeszenie jesd soodkie", 0]], columns=["text", "result"])
# oppinions = preprocess(test_df)
oppinions = preprocess(df_polemo_official)
oppinions.to_csv("data/opinions_hotels_preprocessed.csv", sep=';')

śniadań -> nadań
otrzymali śmy -> otrzymali my
że -> ze
Hyrny -> tyrny
łazienka -> lazienka
No -> do
by m -> by m